In [2]:
from src import API

API.DataAPI.update()
API.DataAPI.prepare_train_data()
API.Predictor.update_factors()

Update Files
Thu May 23 07:03:12 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.01 Secs
Thu May 23 07:03:12 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.03 Secs
Thu May 23 07:03:12 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.01 Secs
Thu May 23 07:03:14 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.33 Secs
Thu May 23 07:03:18 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.30 Secs
Thu May 23 07:03:30 2024 : download since!
Connection and Factor preparation finished!
Thu May 23 07:03:30 2024 : sellside/haitong.hf_factors from 20240523 to 20240523, total 1 periods
Start sellside/haitong.hf_factors:20240523-20240523 
Done sellside/haitong.hf_factors:20240523-20240523, cost 13.8 Secs
Thu May 23 07:03:43 2024 : sellside/haitong.dl_factors from 20240523 to 20240523, total 1 periods
Start sellside/haitong.dl_factors:20240523-20240523 
Done sellside/

In [2]:
from src import API
API.ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
